In [1]:
from darli.robots import RobotModel
from robot_descriptions import ur5_description
import casadi as cs
model = RobotModel(ur5_description.URDF_PATH)

In [2]:
nq = model.nq # dimensionality of configuration  
nv = model.nv # dimensionality of generilized velocities
nu = model.nu # dimensionality  of control inputs
q_min, q_max = model.q_min, model.q_max # minimal and maximal limits on joint pos
nq, nv, nu

(6, 6, 6)

In [3]:
nb = len(model._model._kindyn.joint_names()) - 1

In [4]:
parameters = cs.SX.sym('p', nb*10)

In [5]:
# parameters

In [6]:
joint_regressor = model._model._kindyn.jointTorqueRegressor()

In [7]:
tau = joint_regressor(model._q, model._v, model._dv) @ parameters

In [8]:
# tau

In [9]:
tau = joint_regressor(model._q, model._v, model._dv) @ parameters

inverse_dynamics = cs.Function("inverse_dynamics",
                                [model._q, model._v, model._dv, parameters],
                                [tau],
                                ["q", "v", "dv", "parameters"],
                                ["tau"],)

In [10]:
import numpy as np

In [11]:
inertia = cs.SX.zeros(nv, nv)
unit_vectors = np.eye(nv)
for i in range(nv):
    unit_vector = unit_vectors[i,:]
    inertia[:, i] = joint_regressor(model._q, np.zeros(nv), unit_vector) @ parameters
    
inertia_matrix  = cs.Function("inertia_matrix",
                                [model._q, parameters],
                                [inertia],
                                ["q", "parameters"],
                                ["inertia_matrix"],)

In [12]:
inertia_matrix

Function(inertia_matrix:(q[6],parameters[60])->(inertia_matrix[6x6]) SXFunction)

In [13]:
bias = joint_regressor(model._q, model._v, np.zeros(nv)) @ parameters

bias_term = cs.Function("bias_force",
                                [model._q, model._v, parameters],
                                [bias],
                                ["q", "v", "parameters"],
                                ["bias_force"],)

In [14]:
bias_term

Function(bias_force:(q[6],v[6],parameters[60])->(bias_force[6]) SXFunction)

In [15]:
ddq = cs.solve(inertia, model._u - bias)

forward_dynamics = cs.Function("forward_dynamics",
                                [model._q, model._v, model._u, parameters],
                                [ddq],
                                ["q", "v", "u", "parameters"],
                                ["ddq"],)